In [1]:
%cd /Users/taozang/Zillow/Extracted_Data_MIAMI/12_ZAsmt_CSV/
import pandas as pd
import numpy as np
# tables = ['Building', 'BuildingAreas', 
#           'CareOfName', 'ExteriorWall', 'ExtraFeature', 'Garage', 'InteriorFlooring', 
#           'InteriorWall', 'LotSiteAppeal', 'MailAddress', 'Main', 'Name', 'Oby', 'Pool', 
#           'SaleData', 'TaxDistrict', 'TaxExemption', 'TypeConstruction', 'Value', 'VestingCodes']

import ReadZTRAX as rz
rz.tic()
full_df = pd.read_csv('/Users/taozang/Zillow/Extracted_Data_MIAMI/12_ZAsmt_CSV/Main.csv', low_memory=False)
rz.toc()

/Users/taozang/Zillow/Extracted_Data_MIAMI/12_ZAsmt_CSV
Elapsed time is 14.85540509223938 seconds, or 0.2475902001063029 minutes.


In [2]:
# Step 1: make a deep copy of full_df and remove NaN in Address column
df = full_df.copy() 
df = df.dropna(subset=['PropertyFullStreetAddress']).reset_index()

# Step 2: Find occurances of each unique address
repeated_times = df['PropertyFullStreetAddress'].value_counts() 

# Step3: Find the index for each unique address' first appearance in the column
unique_index = df.drop_duplicates(subset = ["PropertyFullStreetAddress"]).index.values

In [4]:
# Core Algorithm for fast data extraction
# Running time reduced to less than 2 min!
address_dict = {}
rz.tic()
for first_occur_idx in unique_index:
    curr_address = df.loc[first_occur_idx]['PropertyFullStreetAddress']
    occurance = repeated_times[curr_address]
    repeated_subset = df.iloc[first_occur_idx:first_occur_idx + occurance]
    rowID_set = repeated_subset['RowID'].to_list() # use to_list for later saving as json 
    address_dict[curr_address] = rowID_set
rz.toc()

Elapsed time is 88.47343420982361 seconds, or 1.4745573361714681 minutes.


In [10]:
len(address_dict['92 SW 3rd St'])
# in reality this is a large apartment, with over 500 units! 

535

In [6]:
# load json module
import json

# create json object from dictionary
json = json.dumps(address_dict)

# open file for writing, "w" 
f = open("Florida_Miami_address_dictionary.json","w")

# write json object to file
f.write(json)

# close file
f.close()